In [1]:
import pybedtools as pbt
import pysam
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os.path as op

In [3]:
import splanl.junction_scorer as jn
import splanl.merge_bcs as mbcs
import splanl.coords as coords
import splanl.plots as sp
import splanl.score_motifs as sm
import splanl.inspect_variants as iv
import splanl.post_processing as pp

Using TensorFlow backend.


In [4]:
fa_file = '/nfs/kitzman2/jacob/proj/jensplice/20220415_wt1_mpsa_trial3/jkp1053_1054_1055.fa'

In [5]:
refseq = pp.get_refseq( fa_file )

In [11]:
bam1 = ! ls /nfs/kitzman2/jacob/proj/jensplice/20220415_wt1_mpsa_trial3/process_star/BB_test*/*out.wbcs.bam

In [19]:
bam2 = ! ls /nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_RT*/*out.wbcs.bam

In [21]:
bam3 = ! ls /nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_test*/*out.wbcs.bam

In [22]:
bam1

['/nfs/kitzman2/jacob/proj/jensplice/20220415_wt1_mpsa_trial3/process_star/BB_test_Cos1053_JKLab0340_MM1B/BB_test_Cos1053_JKLab0340_MM1BAligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220415_wt1_mpsa_trial3/process_star/BB_test_Cos1053_JKLab0340_MM2B/BB_test_Cos1053_JKLab0340_MM2BAligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220415_wt1_mpsa_trial3/process_star/BB_test_Cos1053_oligodT_MM1B/BB_test_Cos1053_oligodT_MM1BAligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220415_wt1_mpsa_trial3/process_star/BB_test_Cos1053_oligodT_MM2B/BB_test_Cos1053_oligodT_MM2BAligned.out.wbcs.bam']

In [23]:
bam2

['/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_RT_1_340_243_Cos1053/BB_RT_1_340_243_Cos1053Aligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_RT_1_340_243_Hek1053/BB_RT_1_340_243_Hek1053Aligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_RT_1_340_335_Cos1053/BB_RT_1_340_335_Cos1053Aligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_RT_1_340_335_Hek1053/BB_RT_1_340_335_Hek1053Aligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_RT_1b_340_243_Hek1053/BB_RT_1b_340_243_Hek1053Aligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_RT_1b_340_335_Hek1053/BB_RT_1b_340_335_Hek1053Aligned.out.wbcs.bam',
 '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/p

In [24]:
bam = bam1 + bam2 + bam3

In [27]:
msamp_fn = { b.split( '/')[ -1 ].split( '.' )[ 0 ].replace( 'Aligned', '' ) : b for b in bam }

In [28]:
msamp_fn

{'BB_test_Cos1053_JKLab0340_MM1B': '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_test_Cos1053_JKLab0340_MM1B/BB_test_Cos1053_JKLab0340_MM1BAligned.out.wbcs.bam',
 'BB_test_Cos1053_JKLab0340_MM2B': '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_test_Cos1053_JKLab0340_MM2B/BB_test_Cos1053_JKLab0340_MM2BAligned.out.wbcs.bam',
 'BB_test_Cos1053_oligodT_MM1B': '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_test_Cos1053_oligodT_MM1B/BB_test_Cos1053_oligodT_MM1BAligned.out.wbcs.bam',
 'BB_test_Cos1053_oligodT_MM2B': '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_test_Cos1053_oligodT_MM2B/BB_test_Cos1053_oligodT_MM2BAligned.out.wbcs.bam',
 'BB_RT_1_340_243_Cos1053': '/nfs/kitzman2/jacob/proj/jensplice/20220422_wt1_mpsa_trial4and3_deep/process_star/BB_RT_1_340_243_Cos1053/BB_RT_1_340_243_Cos1053Aligned.out.wbcs.bam',
 'BB_RT_1_340_243_Hek10

In [29]:
msamp_rnabam = { samp: pysam.AlignmentFile( msamp_fn[ samp ], 'rb' ) for samp in msamp_fn }

In [30]:
%%time
#I made this more stringent than MSH2 since there is less skipping
#requiring 90 forward and 70 reverse matches
isos_dfs = { samp: jn.get_all_isoforms_pe( msamp_rnabam[ samp ],
                                           [ ( 649, 696 ), ( 3478, 3533 ) ],
                                            spl_tol = 3,
                                            indel_tol = 20,
                                            min_matches_for = 90,
                                            min_matches_rev = 70 )
             for samp in msamp_rnabam }

CPU times: user 25min 49s, sys: 13.6 s, total: 26min 2s
Wall time: 26min 5s


In [31]:
for samp in isos_dfs.keys():
    
    print( samp )
    
    print( isos_dfs[ samp ].head() )

BB_test_Cos1053_JKLab0340_MM1B
                 read_count
isoform                    
((1267, 1359),)     2398582
((1267, 1350),)     2199619
()                    26244
((1284, 1359),)       17744
((1284, 1350),)       12115
BB_test_Cos1053_JKLab0340_MM2B
                 read_count
isoform                    
((1267, 1359),)     5042968
((1267, 1350),)     4600783
()                    43310
((1284, 1359),)       39501
((1284, 1350),)       23727
BB_test_Cos1053_oligodT_MM1B
                 read_count
isoform                    
((1267, 1359),)     2188370
((1267, 1350),)     1959928
()                    23946
((1284, 1359),)       17534
((1284, 1350),)       11526
BB_test_Cos1053_oligodT_MM2B
                 read_count
isoform                    
((1267, 1359),)     6167847
((1267, 1350),)     5723474
()                    53874
((1284, 1359),)       44962
((1284, 1350),)       30211
BB_RT_1_340_243_Cos1053
                 read_count
isoform                    
((1267, 1359),) 

In [32]:
%%time
isogrp_df = jn.number_and_merge_isoforms( isos_dfs )

BB_test_Cos1053_JKLab0340_MM1B
BB_test_Cos1053_JKLab0340_MM2B
BB_test_Cos1053_oligodT_MM1B
BB_test_Cos1053_oligodT_MM2B
BB_RT_1_340_243_Cos1053
BB_RT_1_340_243_Hek1053
BB_RT_1_340_335_Cos1053
BB_RT_1_340_335_Hek1053
BB_RT_1b_340_243_Hek1053
BB_RT_1b_340_335_Hek1053
BB_RT_1b_ODT_243_Hek1053
BB_RT_1b_ODT_335_Hek1053
BB_RT_1_ODT_243_Cos1053
BB_RT_1_ODT_243_Hek1053
BB_RT_1_ODT_335_Cos1053
BB_RT_1_ODT_335_Hek1053
BB_RT_2_340_243_Cos1054
BB_RT_2_340_243_Hek1054
BB_RT_2_340_335_Cos1054
BB_RT_2_340_335_Hek1054
BB_RT_2_ODT_243_Cos1054
BB_RT_2_ODT_243_Hek1054
BB_RT_2_ODT_335_Cos1054
BB_RT_2_ODT_335_Hek1054
BB_RT_3_340_243_Cos1055
BB_RT_3_340_243_Hek1055
BB_RT_3_340_335_Cos1055
BB_RT_3_340_335_Hek1055
BB_RT_3_ODT_243_Cos1055
BB_RT_3_ODT_243_Hek1055
BB_RT_3_ODT_335_cos1055
BB_RT_3_ODT_335_Hek1055
CPU times: user 15 s, sys: 18 ms, total: 15 s
Wall time: 15 s


In [33]:
isogrp_df.head()

,isoform,BB_test_Cos1053_JKLab0340_MM1B_read_count,BB_test_Cos1053_JKLab0340_MM2B_read_count,BB_test_Cos1053_oligodT_MM1B_read_count,BB_test_Cos1053_oligodT_MM2B_read_count,BB_RT_1_340_243_Cos1053_read_count,BB_RT_1_340_243_Hek1053_read_count,BB_RT_1_340_335_Cos1053_read_count,BB_RT_1_340_335_Hek1053_read_count,BB_RT_1b_340_243_Hek1053_read_count,...,BB_RT_2_ODT_335_Cos1054_read_count,BB_RT_2_ODT_335_Hek1054_read_count,BB_RT_3_340_243_Cos1055_read_count,BB_RT_3_340_243_Hek1055_read_count,BB_RT_3_340_335_Cos1055_read_count,BB_RT_3_340_335_Hek1055_read_count,BB_RT_3_ODT_243_Cos1055_read_count,BB_RT_3_ODT_243_Hek1055_read_count,BB_RT_3_ODT_335_cos1055_read_count,BB_RT_3_ODT_335_Hek1055_read_count
isonum,,,,,,,,,,,,,,,,,,,,,
iso0000,"((1136, 1225), (1301, 1350))",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
iso0001,"((1267, 1290), (1320, 1350))",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
iso0002,"((1267, 1314), (1337, 1359))",1,272,0,0,1,127,64,59,140,...,0,0,0,0,0,0,0,0,0,0
iso0003,"((1267, 1282), (1328, 1359))",0,0,2,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
iso0004,"((1136, 1224), (1306, 1359))",0,0,0,0,0,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [34]:
isogrp_df.loc[ isogrp_df.isoform == ((1267, 1359),) ]

,isoform,BB_test_Cos1053_JKLab0340_MM1B_read_count,BB_test_Cos1053_JKLab0340_MM2B_read_count,BB_test_Cos1053_oligodT_MM1B_read_count,BB_test_Cos1053_oligodT_MM2B_read_count,BB_RT_1_340_243_Cos1053_read_count,BB_RT_1_340_243_Hek1053_read_count,BB_RT_1_340_335_Cos1053_read_count,BB_RT_1_340_335_Hek1053_read_count,BB_RT_1b_340_243_Hek1053_read_count,...,BB_RT_2_ODT_335_Cos1054_read_count,BB_RT_2_ODT_335_Hek1054_read_count,BB_RT_3_340_243_Cos1055_read_count,BB_RT_3_340_243_Hek1055_read_count,BB_RT_3_340_335_Cos1055_read_count,BB_RT_3_340_335_Hek1055_read_count,BB_RT_3_ODT_243_Cos1055_read_count,BB_RT_3_ODT_243_Hek1055_read_count,BB_RT_3_ODT_335_cos1055_read_count,BB_RT_3_ODT_335_Hek1055_read_count
isonum,,,,,,,,,,,,,,,,,,,,,
iso2285,"((1267, 1359),)",2398582,5042968,2188370,6167847,1848512,3402477,498991,1002709,3088366,...,147916,670428,933746,2231420,641615,1423772,2632227,1602454,768075,1087995


In [35]:
isogrp_df.loc[ isogrp_df.isoform == ((1267, 1350),) ]

,isoform,BB_test_Cos1053_JKLab0340_MM1B_read_count,BB_test_Cos1053_JKLab0340_MM2B_read_count,BB_test_Cos1053_oligodT_MM1B_read_count,BB_test_Cos1053_oligodT_MM2B_read_count,BB_RT_1_340_243_Cos1053_read_count,BB_RT_1_340_243_Hek1053_read_count,BB_RT_1_340_335_Cos1053_read_count,BB_RT_1_340_335_Hek1053_read_count,BB_RT_1b_340_243_Hek1053_read_count,...,BB_RT_2_ODT_335_Cos1054_read_count,BB_RT_2_ODT_335_Hek1054_read_count,BB_RT_3_340_243_Cos1055_read_count,BB_RT_3_340_243_Hek1055_read_count,BB_RT_3_340_335_Cos1055_read_count,BB_RT_3_340_335_Hek1055_read_count,BB_RT_3_ODT_243_Cos1055_read_count,BB_RT_3_ODT_243_Hek1055_read_count,BB_RT_3_ODT_335_cos1055_read_count,BB_RT_3_ODT_335_Hek1055_read_count
isonum,,,,,,,,,,,,,,,,,,,,,
iso1188,"((1267, 1350),)",2199619,4600783,1959928,5723474,1639853,3024211,456880,942554,2722208,...,131835,643764,853122,1925365,606692,1466165,2361296,1371133,728819,1187498


In [36]:
satbls = ! ls /nfs/turbo/umms-kitzmanj/oldvol2/jacob/proj/jensplice/20220426_wt1_subasm_filter_stringent/sapipe/sa/*.haps.all.txt

In [37]:
satbl_fn = { sa.split( '/')[ -1 ].split( '.' )[ 0 ]: pd.read_table( sa ).set_index( 'readgroupid' ) for sa in satbls }

In [38]:
for lib in satbl_fn.keys():
    
    satbl_fn[ lib ] = satbl_fn[ lib ][ [ col for col in satbl_fn[ lib ] if not col.startswith( 'nbp_' ) ] ].copy()

In [39]:
exonbed = pbt.BedTool( '/nfs/kitzman2/smithcat/proj/wt1_2022/refs/wt1_ex9.bed' )

In [40]:
isos = jn.make_junction_graph( exonbed )

In [41]:
isos

{'iso00': ((650, 696), (1267, 1350), (3479, 3533)),
 'iso01': ((650, 696), (1267, 1359), (3479, 3533)),
 'iso02': ((650, 696), (3479, 3533))}

In [42]:
unique_jns = list( { jn for grp,jn_tups in isos.items() for jn_tup in jn_tups for jn in jn_tup
                       if 696 < jn < 3479 } ) 

In [43]:
unique_jns

[1267, 1350, 1359]

In [44]:
msamp_rnabam = { samp : pysam.AlignmentFile( msamp_fn[ samp ], 'rb' ) for samp in msamp_fn }

In [48]:
for lib in satbl_fn:
    
    print( lib )
    
    print( satbl_fn[ lib ].head() )

JKP1053
                         passes                              status  \
readgroupid                                                           
AAAAAAAATCACGACCCTCCTGG   False  possible_chimeric_no_major_variant   
AAAAAAAGGGCTTCCGGGTATGG    True                   no_variants_input   
AAAAAAAGGGTTCCAGACTGTGG   False                     toomanymajorvar   
AAAAAAAGTAGTCTGGTGTGTGG    True                                pass   
AAAAAAATACCCCGGATGATTGG   False  possible_chimeric_no_major_variant   

                         n_variants_passing  \
readgroupid                                   
AAAAAAAATCACGACCCTCCTGG                   6   
AAAAAAAGGGCTTCCGGGTATGG                   0   
AAAAAAAGGGTTCCAGACTGTGG                   2   
AAAAAAAGTAGTCTGGTGTGTGG                   1   
AAAAAAATACCCCGGATGATTGG                   1   

                                                              variant_list  
readgroupid                                                                 
AAAAAAAATC

In [ ]:
%%time
#18 min/sample

iso_df_stats = { lib: jn.summarize_isos_by_var_bc_pe( { samp: bam for samp,bam in msamp_rnabam.items()
                                                        if lib.replace( 'JKP', '' ) in samp },
                                                      [ ( 649, 696 ), ( 3478, 3533 ) ],
                                                      satbl_fn[ lib ],
                                                      isogrp_df,
                                                      unique_jns,
                                                      [ ( ( 1266, 1350 ), ), ( ( 1266, 1359 ), ), () ],
                                                      spl_tol = 3,
                                                      indel_tol = 20,
                                                      min_matches_for = 90,
                                                      min_matches_rev = 70,
                                                      bc_tag = 'BC',
                                                     ) 
                  for lib in satbl_fn }

BB_test_Cos1053_JKLab0340_MM1B
Barcodes processed: 1000
Reads processed: 93024
Barcodes processed: 2000
Reads processed: 191682
Barcodes processed: 3000
Reads processed: 273931
Barcodes processed: 4000
Reads processed: 364737
Barcodes processed: 5000
Reads processed: 451756
Barcodes processed: 6000
Reads processed: 544347
Barcodes processed: 7000
Reads processed: 632880
Barcodes processed: 8000
Reads processed: 722562
Barcodes processed: 9000
Reads processed: 820945
Barcodes processed: 10000
Reads processed: 911421
Barcodes processed: 11000
Reads processed: 1016529
Barcodes processed: 12000
Reads processed: 1106018
Barcodes processed: 13000
Reads processed: 1196871
Barcodes processed: 14000
Reads processed: 1279955
Barcodes processed: 15000
Reads processed: 1399232
Barcodes processed: 16000
Reads processed: 1483319
Barcodes processed: 17000
Reads processed: 1580047
Barcodes processed: 18000
Reads processed: 1660821
Barcodes processed: 19000
Reads processed: 1748885
Barcodes processed: 

For isoform: ((1269, 1350),)
The variants with the top 5 number of barcodes are:
[('jkp815:1267:GTG:GG,jkp815:1365:T:C', 8), ('jkp815:1267:GTG:GG', 6), ('jkp815:1288:A:C', 3), ('jkp815:1269:G:T', 3), ('jkp815:1360:G:T', 3)]
For isoform: ((1267, 1274), (1296, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1273:AACCATTCCAGTGTAAAACTTGT:AT,jkp815:1316:C:T', 2)]
For isoform: ((1146, 1231), (1317, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1224:CCAC:CC,jkp815:1296:C:T,jkp815:1353:A:G', 1), ('jkp815:1312:G:T,jkp815:1392:T:A', 1), ('jkp815:1227:C:T', 1)]
For isoform: ((1267, 1312), (1351, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1252:T:G', 2), ('jkp815:1345:A:T', 1), ('jkp815:1327:A:T', 1), ('jkp815:1368:A:T', 1), ('jkp815:1339:G:T', 1)]
For isoform: ((1267, 1327),)
The variants with the top 5 number of barcodes are:
[('jkp815:1277:A:C,jkp815:1309:C:G,jkp815:1337:C:T,jkp815:1343:T:A', 1), ('jkp815:1227:C:A,jkp815:1316:

For isoform: ((1267, 1284), (1318, 1350))
The variants with the top 5 number of barcodes are:
[('jkp815:1228:A:C', 2), ('jkp815:1228:A:G', 2), ('jkp815:1366:A:C', 2), ('jkp815:1318:A:T', 2), ('jkp815:1345:A:G', 2)]
For isoform: ((1267, 1355), (2705, 2710))
The variants with the top 5 number of barcodes are:
[('jkp815:1352:TAAAACAAG:TAAAAACAAG', 1)]
For isoform: ((1273, 1359),)
The variants with the top 5 number of barcodes are:
[('jkp815:1246:A:C', 2), ('jkp815:1389:T:A', 2), ('jkp815:1310:C:G', 1), ('jkp815:1227:C:G', 1), ('jkp815:1283:G:C', 1)]
For isoform: ((1267, 1322), (1353, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1307:C:G', 1), ('jkp815:1353:A:C', 1), ('jkp815:1369:CTTTTCTTCA:CTTTCTTCA,jkp815:1381:T:A', 1), ('jkp815:1383:T:A', 1), ('jkp815:1246:AGA:AA', 1)]
For isoform: ((1267, 1304), (1336, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1287:A:T', 1), ('jkp815:1358:A:G', 1), ('jkp815:1373:TCT:TT,jkp815:1386:T:G', 1), ('jkp815:1

For isoform: ((1267, 1362),)
The variants with the top 5 number of barcodes are:
[('jkp815:1312:G:C', 1)]
For isoform: ((1267, 1285), (1333, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1393:T:C', 1), ('jkp815:1259:C:G,jkp815:1273:A:T', 1), ('jkp815:1280:C:G', 1), ('jkp815:1322:CCTGAAGAC:CC', 1), ('jkp815:1306:T:G', 1)]
For isoform: ((1136, 1359),)
The variants with the top 5 number of barcodes are:
[('jkp815:1227:CAT:CT,jkp815:1294:G:A,jkp815:1308:T:C,jkp815:1330:C:T', 1), ('jkp815:1219:T:C,jkp815:1234:A:T', 1), ('jkp815:1360:G:C', 1), ('jkp815:1301:A:T', 1), ('jkp815:1349:A:G', 1)]
For isoform: ((1267, 1282), (1316, 1350))
The variants with the top 5 number of barcodes are:
[('jkp815:1385:T:G', 1), ('jkp815:1279:T:C', 1), ('jkp815:1367:A:G', 1)]
For isoform: ((1267, 1355), (2483, 2488))
The variants with the top 5 number of barcodes are:
[('jkp815:1218:C:T,jkp815:1253:C:G', 1)]
For isoform: ((1267, 1344), (2393, 2399))
The variants with the top 5 number of bar

For isoform: ((1267, 1319), (1341, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1367:A:G', 1), ('jkp815:1334:C:A', 1), ('jkp815:1311:C:A', 1), ('jkp815:1314:TCCGA:TCGA,jkp815:1321:A:C', 1)]
For isoform: ((1267, 1351), (2772, 2780))
The variants with the top 5 number of barcodes are:
[('jkp815:1297:A:T', 1)]
For isoform: ((1267, 1343), (2166, 2174))
The variants with the top 5 number of barcodes are:
[('jkp815:1344:CATACAGGT:CT', 1)]
For isoform: ((1267, 1361),)
The variants with the top 5 number of barcodes are:
[('jkp815:1311:C:T,jkp815:1368:A:C', 1), ('jkp815:1352:T:C', 1)]
For isoform: ((1236, 1262), (1320, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1236:G:T,jkp815:1248:A:C,jkp815:1392:T:A', 1), ('jkp815:1299:C:A', 1)]
For isoform: ((1190, 1224), (1296, 1350))
The variants with the top 5 number of barcodes are:
[('jkp815:1317:G:C', 1)]
For isoform: ((1267, 1315), (3201, 3213))
The variants with the top 5 number of barcodes are:
[('jk

...

Barcodes processed: 165000
Reads processed: 798995
Barcodes processed: 166000
Reads processed: 802670
Barcodes processed: 167000
Reads processed: 806638
Barcodes processed: 168000
Reads processed: 810778
Barcodes processed: 169000
Reads processed: 814303
For isoform: ((1267, 1350),)
The variants with the top 5 number of barcodes are:
[('jkp815:1227:C:A', 1164), ('jkp815:1227:C:T', 1125), ('jkp815:1227:C:G', 900), ('jkp815:1228:A:C', 753), ('jkp815:1228:A:G', 656)]
For isoform: ((1267, 1359),)
The variants with the top 5 number of barcodes are:
[('jkp815:1227:C:A', 1254), ('jkp815:1227:C:T', 1185), ('jkp815:1227:C:G', 1001), ('jkp815:1228:A:C', 811), ('jkp815:1228:A:G', 706)]
For isoform: ((1284, 1359),)
The variants with the top 5 number of barcodes are:
[('jkp815:1266:G:T', 100), ('jkp815:1266:G:C', 60), ('jkp815:1267:G:T', 39), ('jkp815:1266:G:A', 36), ('jkp815:1258:T:A', 35)]
For isoform: ()
The variants with the top 5 number of barcodes are:
[('jkp815:1265:A:G', 42), ('jkp815:1265:

For isoform: ((1000, 1359),)
The variants with the top 5 number of barcodes are:
[('jkp815:1282:A:G', 1), ('jkp815:1259:C:T,jkp815:1304:G:T,jkp815:1354:A:T', 1), ('jkp815:1292:T:C,jkp815:1341:A:T', 1), ('jkp815:1273:A:C,jkp815:1297:A:G,jkp815:1339:G:C', 1), ('jkp815:1212:G:A', 1)]
For isoform: ((1267, 1328), (1354, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1274:A:T,jkp815:1328:GACCCACACCAGGACTCATACAGGTAA:GA', 11), ('jkp815:1328:GACCCACACCAGGACTCATACAGGTAA:GA,jkp815:1395:T:G', 6), ('jkp815:1227:C:A', 2), ('jkp815:1242:G:T,jkp815:1274:A:T,jkp815:1328:GACCCACACCAGGACTCATACAGGTAA:GA', 1), ('jkp815:1260:C:G,jkp815:1395:T:G', 1)]
For isoform: ((1190, 1230), (1299, 1350))
The variants with the top 5 number of barcodes are:
[('jkp815:1299:C:G', 1)]
For isoform: ((1267, 1295), (1337, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1295:TCAGCGAAAGTTCTCCCGGTCCGACCACCTGAAGACCCACACC:TC', 2), ('jkp815:1304:GTTCTCCCGGTCCGACCACCTGAAGACCCACAC:GC', 1)]
For

For isoform: ((1267, 1312), (1334, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1298:GCGAAAGTTCTCCCGGTCCGACCACCTGAAGACCCACACC:GC,jkp815:1338:A:C', 2), ('jkp815:1227:C:T,jkp815:1240:G:T', 1), ('jkp815:1331:C:G', 1), ('jkp815:1301:A:C', 1)]
For isoform: ((1313, 1359),)
The variants with the top 5 number of barcodes are:
[('jkp815:1206:T:C,jkp815:1308:T:C', 1), ('jkp815:1219:T:A,jkp815:1221:TGC:TGAGGC', 1), ('jkp815:1280:C:T,jkp815:1340:G:A', 1), ('jkp815:1368:A:T', 1), ('jkp815:1225:C:T,jkp815:1243:GCT:GT,jkp815:1269:G:T,jkp815:1393:T:G', 1)]
For isoform: ((1267, 1321), (1344, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1321:ACCTGAAGACCCACACCAGGACTC:AC', 5), ('jkp815:1274:A:T,jkp815:1321:ACCTGAAGACCCACACCAGGACTC:AC', 1), ('jkp815:1313:G:A,jkp815:1321:ACCTGAAGACCCACACCAGGACTC:AC,jkp815:1321:ACCTGAAGACCCACACCAGGACTC:ACCTGAGGACCCACACCAGGACTC', 1), ('jkp815:1321:ACCTGAAGACCCACACCAGGACTC:AC,jkp815:1351:G:A,jkp815:1364:G:C,jkp815:1398:T:A', 1), 

For isoform: ((1267, 1287), (1338, 1359))
The variants with the top 5 number of barcodes are:
[('jkp815:1287:AAAACTTGTCAGCGAAAGTTCTCCCGGTCCGACCACCTGAAGACCCACACCA:AA', 1), ('jkp815:1262:T:G,jkp815:1368:A:C', 1)]
For isoform: ((1236, 1242), (1293, 1350))
The variants with the top 5 number of barcodes are:
[('jkp815:1338:A:C,jkp815:1363:C:G', 1)]
For isoform: ((1293, 1340), (1893, 1898))
The variants with the top 5 number of barcodes are:
[('jkp815:1258:T:C,jkp815:1307:C:T,jkp815:1341:A:G,jkp815:1343:TCA:TA', 1)]
For isoform: ((1170, 1225), (1289, 1350))
The variants with the top 5 number of barcodes are:
[('jkp815:1228:ATTGTTA:ATGTTA,jkp815:1228:ATTGTTA:ATTGTTC,jkp815:1391:ATTATTTTT:ATATTTTT', 1)]
For isoform: ((1267, 1296), (1341, 1350))
The variants with the top 5 number of barcodes are:
[('jkp815:1234:A:T,jkp815:1296:CAGCGAAAGTTCTCCCGGTCCGACCACCTGAAGACCCACACCAGGA:CA', 1)]
For isoform: ((1236, 1242), (1289, 1350))
The variants with the top 5 number of barcodes are:
[('jkp815:1309:CCCGG

Barcodes processed: 1000
Reads processed: 10813
Barcodes processed: 2000
Reads processed: 21948
Barcodes processed: 3000
Reads processed: 32149
Barcodes processed: 4000
Reads processed: 44233
Barcodes processed: 5000
Reads processed: 55762
Barcodes processed: 6000
Reads processed: 67666
Barcodes processed: 7000
Reads processed: 79990
Barcodes processed: 8000
Reads processed: 90167
Barcodes processed: 9000
Reads processed: 100461
Barcodes processed: 10000
Reads processed: 110975
Barcodes processed: 11000
Reads processed: 122282
Barcodes processed: 12000
Reads processed: 132138
Barcodes processed: 13000
Reads processed: 142915
Barcodes processed: 14000
Reads processed: 154834
Barcodes processed: 15000
Reads processed: 167551
Barcodes processed: 16000
Reads processed: 181413
Barcodes processed: 17000
Reads processed: 195561
Barcodes processed: 18000
Reads processed: 209350
Barcodes processed: 19000
Reads processed: 222475
Barcodes processed: 20000
Reads processed: 236337
Barcodes processe

In [62]:
for lib in iso_df_stats:
    print( lib, iso_df_stats[ lib ].shape )

JKP1053 (6354, 117)
JKP1054 (6354, 77)
JKP1055 (6354, 77)


In [63]:
for lib in iso_df_stats:
    print( lib, iso_df_stats[ lib ].query( 'total_passfilt > 0' ).shape )

JKP1053 (227, 117)
JKP1054 (224, 77)
JKP1055 (257, 77)


In [64]:
for lib in iso_df_stats:
    print( lib, ( iso_df_stats[ lib ].query( 'total_passfilt == 0' ).total_read_count.sum() / iso_df_stats[ lib ].total_read_count.sum() )*100 )

JKP1053 0.24681957544293046
JKP1054 0.40728994447198286
JKP1055 0.39886035242943624


In [65]:
for lib in iso_df_stats:
    print( lib, ( iso_df_stats[ lib ].query( 'total_passfilt == 0' ).total_num_bcs.sum() / iso_df_stats[ lib ].total_num_bcs.sum() )*100 )

JKP1053 0.31062184024107364
JKP1054 0.36252133322385294
JKP1055 0.20012527710218403


In [66]:
iso_df_stats[ 'JKP1053' ]

,isoform,BB_test_Cos1053_JKLab0340_MM1B_read_count,BB_test_Cos1053_JKLab0340_MM2B_read_count,BB_test_Cos1053_oligodT_MM1B_read_count,BB_test_Cos1053_oligodT_MM2B_read_count,BB_RT_1_340_243_Cos1053_read_count,BB_RT_1_340_243_Hek1053_read_count,BB_RT_1_340_335_Cos1053_read_count,BB_RT_1_340_335_Hek1053_read_count,BB_RT_1b_340_243_Hek1053_read_count,...,BB_RT_1_ODT_335_Cos1053_filter,BB_RT_1_ODT_335_Hek1053_num_bcs,BB_RT_1_ODT_335_Hek1053_num_vars,BB_RT_1_ODT_335_Hek1053_max_reads_per_bc,BB_RT_1_ODT_335_Hek1053_max_bc_per_var,BB_RT_1_ODT_335_Hek1053_filter,total_read_count,total_num_bcs,total_num_vars,total_passfilt
isonum,,,,,,,,,,,,,,,,,,,,,
iso0000,"((1136, 1225), (1301, 1350))",0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,1.0,1.0,0
iso0001,"((1267, 1290), (1320, 1350))",0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,1.0,1.0,0
iso0002,"((1267, 1314), (1337, 1359))",1,272,0,0,1,127,64,59,140,...,0.0,10.0,6.0,7.0,5.0,3.0,1105,170.0,129.0,9
iso0003,"((1267, 1282), (1328, 1359))",0,0,2,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,11,2.0,2.0,0
iso0004,"((1136, 1224), (1306, 1359))",0,0,0,0,0,1,0,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,4,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iso6349,"((1236, 1242), (1266, 1350))",0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,0
iso6350,"((1146, 1222), (1281, 1359))",0,0,0,0,0,3,0,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,12,1.0,1.0,0
iso6351,"((1284, 1348),)",0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,1.0,1.0,0


In [ ]:
bdout = '/nfs/kitzman2/smithcat/proj/wt1_2022/ex9_data/'

In [ ]:
for lib in iso_df_stats.keys():    
    iso_df_stats[ lib ].reset_index().to_csv( bdout + 'wt1_ex9_%s_isoforms_2022-0502.txt' % lib,
                                               sep = '\t',
                                               index = False )

In [76]:
iso_df_stats_all = iso_df_stats[ 'JKP1053' ].merge( iso_df_stats[ 'JKP1054' ],
                                                    left_index = True,
                                                    right_index = True,
                                                    how = 'outer',
                                                    suffixes = ( '_JKP1053', '_JKP1054' )
                                                  )

In [77]:
iso_df_stats_all

,isoform_JKP1053,BB_test_Cos1053_JKLab0340_MM1B_read_count_JKP1053,BB_test_Cos1053_JKLab0340_MM2B_read_count_JKP1053,BB_test_Cos1053_oligodT_MM1B_read_count_JKP1053,BB_test_Cos1053_oligodT_MM2B_read_count_JKP1053,BB_RT_1_340_243_Cos1053_read_count_JKP1053,BB_RT_1_340_243_Hek1053_read_count_JKP1053,BB_RT_1_340_335_Cos1053_read_count_JKP1053,BB_RT_1_340_335_Hek1053_read_count_JKP1053,BB_RT_1b_340_243_Hek1053_read_count_JKP1053,...,BB_RT_2_ODT_335_Cos1054_filter,BB_RT_2_ODT_335_Hek1054_num_bcs,BB_RT_2_ODT_335_Hek1054_num_vars,BB_RT_2_ODT_335_Hek1054_max_reads_per_bc,BB_RT_2_ODT_335_Hek1054_max_bc_per_var,BB_RT_2_ODT_335_Hek1054_filter,total_read_count_JKP1054,total_num_bcs_JKP1054,total_num_vars_JKP1054,total_passfilt_JKP1054
isonum,,,,,,,,,,,,,,,,,,,,,
iso0000,"((1136, 1225), (1301, 1350))",0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0
iso0001,"((1267, 1290), (1320, 1350))",0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0
iso0002,"((1267, 1314), (1337, 1359))",1,272,0,0,1,127,64,59,140,...,0.0,0.0,0.0,0.0,0.0,0.0,1105,0.0,0.0,0
iso0003,"((1267, 1282), (1328, 1359))",0,0,2,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0
iso0004,"((1136, 1224), (1306, 1359))",0,0,0,0,0,1,0,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iso6349,"((1236, 1242), (1266, 1350))",0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0
iso6350,"((1146, 1222), (1281, 1359))",0,0,0,0,0,3,0,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,12,0.0,0.0,0
iso6351,"((1284, 1348),)",0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0


In [78]:
iso_df_stats_all = iso_df_stats_all.merge( iso_df_stats[ 'JKP1055' ].rename( columns = { col: col + '_JKP1055' 
                                                                                         for col in iso_df_stats[ 'JKP1055' ] if col.startswith( 'total_' ) } ),
                                            left_index = True,
                                            right_index = True,
                                            how = 'outer',
                                            suffixes = ( '', '_JKP1055' )
                                        )

In [79]:
total_cols = [ 'total_read_count', 'total_num_bcs', 'total_num_vars', 'total_passfilt' ]

for col in total_cols:
    
    iso_df_stats_all[ col ] = iso_df_stats_all[ [ lcol for lcol in iso_df_stats_all if col + '_' in lcol ] ].sum( axis = 1 )

In [80]:
len( bam )

36

In [81]:
iso_df_stats_all.shape

(6354, 275)

In [82]:
iso_df_stats_all.query( 'total_passfilt > 0' ).shape

(427, 275)

In [93]:
iso_df_stats_all.query( 'total_passfilt >= 16' ).shape

(66, 275)

In [86]:
( iso_df_stats_all.query( 'total_passfilt == 0' ).total_read_count.sum() / iso_df_stats_all.total_read_count.sum() )*100

0.11176785139485684

In [92]:
( iso_df_stats_all.query( 'total_passfilt >= 16' ).total_read_count.sum() / iso_df_stats_all.total_read_count.sum() )*100

99.49083515501987

In [87]:
( iso_df_stats_all.query( 'total_passfilt == 0' ).total_num_bcs.sum() / iso_df_stats_all.total_num_bcs.sum() )*100

0.21927283253478538

In [91]:
( iso_df_stats_all.query( 'total_passfilt >= 16' ).total_num_bcs.sum() / iso_df_stats_all.total_num_bcs.sum() )*100

99.08175328611193

In [90]:
iso_df_stats_all.total_num_bcs.isnull().sum()

0

In [89]:
iso_df_stats_all.reset_index().to_csv( bdout + 'wt1_ex9_isoforms_2022-0502.txt',
                                               sep = '\t',
                                               index = False )